# Building Feature Based Grammar

In [1]:
import nltk

### Grammarical Features

In contrast to feature extractors, which record features that have been automatically detected, we are now going to declare the features of words and phrases. We start off with a very simple example, using dictionaries to store features and their values.

In [8]:
kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF':'k'}
chase = {'CAT':'V', 'ORTH': 'chased', 'REL': 'chase'}

CAT - Grammatical Category<br>
ORTH - Orthography (spelling)<br>
The above are shared features<br>

Semantically-oriented feature: kim['**REF**'] is intended to give the referent of kim, while chase['**REL**'] gives the relation expressed by chase. In the context of rule-based grammars, such pairings of features and values are known as feature **structures**.

Feature structures include various kinds of information about grammatical entities. 

In the case of a verb, one must know what 'semantic role' is played by the arguments of the verb. For example, 'chased', the subject plays the role of 'agent' while object has the role of 'patient'. 

 Let's add this information, using 'sbj' and 'obj' as placeholders which will get filled once the verb combines with its grammatical arguments:

In [9]:
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

If we process 'Kim chased Lee', we want to bind the verb's agent role to the subject and the patient role to the 'object'. We do this by linking to the REF feature of the relevant NP.

In the following example, we make the simple-minded assumption that the NPs immediately to the left and right of the verb are the subject and object respectively. We also add a feature structure for Lee to complete the example.

In [10]:
sent = 'Kim chased Lee'
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}

def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs
        
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])
verb['AGT'] = subj['REF']
verb['PAT'] = obj['REF']

for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print('%-5s ==> %s' % (k, verb[k]))

ORTH  ==> chased
REL   ==> chase
AGT   ==> k
PAT   ==> l


The same approach could be adopted for a different verb, say surprise, though in this case, the subject would play the role of "source" (SRC) and the object, the role of "experiencer" (EXP):

In [11]:
surprise = {'CAT': 'V', 'ORTH': 'surprised', 'REL': 'surprise',
       'SRC': 'sbj', 'EXP': 'obj'}

But this type of construction of feature structures is ad hoc. We want something more generic. 

In [12]:
nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


### Processing Feature Structures

Feature Structures in NLTK are declared with the FeatStruct() constructor. Atomic feature values can be strings or integers

In [13]:
fs1 = nltk.FeatStruct(TENSE = 'past', NUM = 'sg')
print(fs1)

[ NUM   = 'sg'   ]
[ TENSE = 'past' ]


A feature structure is actually just a kind of dictionary, and so we access its values by indexing in the usual way. We can use our familiar syntax to assign values to features:

In [14]:
fs1 = nltk.FeatStruct(PER = 3, NUM = 'pl', GND= 'fm')
fs1['GND']

'fm'

In [15]:
fs1['CASE'] = 'acc'
print(fs1)

[ CASE = 'acc' ]
[ GND  = 'fm'  ]
[ NUM  = 'pl'  ]
[ PER  = 3     ]


We can also define feature structures that have complex values, as discussed earlier

In [16]:
fs2 = nltk.FeatStruct(POS = 'N', AGR = fs1)
print(fs2)

[       [ CASE = 'acc' ] ]
[ AGR = [ GND  = 'fm'  ] ]
[       [ NUM  = 'pl'  ] ]
[       [ PER  = 3     ] ]
[                        ]
[ POS = 'N'              ]


In [18]:
fs2['AGR']

[CASE='acc', GND='fm', NUM='pl', PER=3]

An alternative method of specifying feature structures is to use a bracketed string consisting of feature-value pairs in the format feature=value, where values may themselves be feature structures

In [20]:
nltk.FeatStruct('[POS = "N", AGR = [PER =3, GND = "fem", NUM = "pl"]]')

[AGR=[GND='fem', NUM='pl', PER=3], POS='N']

Feature structures are not inherently tied to linguistic objects; they are general purpose structures for representing knowledge. For example, we could encode information about a person in a feature structure:

In [24]:
nltk.FeatStruct(NAME = 'Lee', TELNO= '012823', AGE = 33)

[AGE=33, NAME='Lee', TELNO='012823']

Feature Structures can be viewed as Directed Acyclic Graphs 

In order to indicate reentrancy in our matrix-style representations, we will prefix the first occurrence of a shared feature structure with an integer in parentheses, such as (1). Any later reference to that structure will use the notation ->(1), as shown below.



In [28]:
f1 = nltk.FeatStruct("""[NAME = 'LEE', ADDRESS =(1)[NUMBER = 74, 
                    STREET = 'RUE PASCAL'], SPOUSE = [NAME = 'KIM', 
                    ADDRESS -> (1)]]""")
print(f1)

[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'RUE PASCAL' ] ]
[                                         ]
[ NAME    = 'LEE'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'KIM' ]           ]


In [29]:
f1['SPOUSE']['ADDRESS']

[NUMBER=74, STREET='RUE PASCAL']

The bracketed integer is sometimes called a tag or a coindex. The choice of integer is not significant. There can be any number of tags within a single feature structure.

In [30]:
print(nltk.FeatStruct("[A = 'a', B=(1)[C= 'c'], D->(1), E->(1)]"))

[ A = 'a'             ]
[                     ]
[ B = (1) [ C = 'c' ] ]
[                     ]
[ D -> (1)            ]
[ E -> (1)            ]


##### Subsumption and Unification

It is standard to think of feature structures as providing partial information about some object, in the sense that we can order feature structures according to how much information they contain.

(23)
a.  [NUMBER =74]

b.  [NUMBER =74]
    [STREET = 'RUE PASCAL]

c. [NUMBER =74]
    [STREET = 'RUE PASCAL]
    [CITY = 'PARIS]
    
This ordering is called **subsumption**. FS0 subsumes FS1 if all information contained in FS0 is also contained in Fs1. We use the symbol ⊑ to represent subsumption.

When we add the possibility of reentrancy, we need to be more careful about how we describe subsumption: if FS0 ⊑ FS1, then FS1 must have all the paths and reentrancies of FS0.

So we have seen that some feature structures carry more information than others. How do we go about adding more information to a given feature structure? For example, we might decide that addresses should consist of not just a street number and a street name, but also a city. 

Merging information from two features is called **unification** and is supported by **unify()** method.

In [32]:
fs1 = nltk.FeatStruct(NUMBER = 74, STREET = 'RUE PASCAL')
fs2 = nltk.FeatStruct(CITY = 'PARIS')
fs1.unify(fs2)

[CITY='PARIS', NUMBER=74, STREET='RUE PASCAL']

In [33]:
fs1

[NUMBER=74, STREET='RUE PASCAL']

Unification is formally defined as a (partial) binary operation: FS0 ⊔ FS1. Unification is symmetric, so FS0 ⊔ FS1 = FS1 ⊔ FS0. The same is true in Python:

In [34]:
fs2.unify(fs1)

[CITY='PARIS', NUMBER=74, STREET='RUE PASCAL']

If we unify two feature structures which stand in the subsumption relationship, then the result of unification is the most informative of the two:

If FS0 ⊑ FS1, then FS0 ⊔ FS1 = FS1

Unification between FS0 and FS1 will fail if the two feature structures share a path π, but the value of π in FS0 is a distinct atom from the value of π in FS1. This is implemented by setting the result of unification to be None.

In [37]:
fs0 = nltk.FeatStruct(A = 'a')
fs1 = nltk.FeatStruct(A = 'b')
print(fs0.unify(fs1))

None


In [38]:
fs2 = nltk.FeatStruct(A = 'a')
print(fs0.unify(fs2))

[ A = 'a' ]


Now, if we look at how unification interacts with structure-sharing, things become really interesting. 

In [44]:
fs0 = nltk.FeatStruct("""[NAME = 'LEE', ADDRESS = [NUMBER = 74, STREET = 'RUE PASCAL'],
                     AGE = 33, SPOUSE = [NAME = 'KIM', ADDRESS = [NUMBER = 74, STREET = 'RUE PASCAL']]]""")
print(fs1)

[ SPOUSE = [ ADDRESS = [ CITY = 'PARIS' ] ] ]


What happens when we augment Kim's address with a specification for CITY? Notice that fs1 needs to include the whole path from the root of the feature structure down to CITY.

In [45]:
fs1 = nltk.FeatStruct('[SPOUSE = [ADDRESS = [CITY = "PARIS"]]]')
print(fs1.unify(fs0))

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'RUE PASCAL' ]               ]
[                                                   ]
[ AGE     = 33                                      ]
[ NAME    = 'LEE'                                   ]
[                                                   ]
[           [           [ CITY   = 'PARIS'      ] ] ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'RUE PASCAL' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'KIM'                     ] ]


By contrast, the result is very different if fs1 is unified with the structure-sharing version fs2

In [48]:
fs2 = nltk.FeatStruct("""[NAME = LEE, ADDRESS = (1)[NUMBER = 74, STREET = 'RUE PASCAL'],
                        SPOUSE = [NAME = KIM, ADDRESS -> (1)]]""")

print(fs1.unify(fs2))

[               [ CITY   = 'PARIS'      ] ]
[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'RUE PASCAL' ] ]
[                                         ]
[ NAME    = 'LEE'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'KIM' ]           ]


Rather than just updating what was in effect Kim's "copy" of Lee's address, we have now updated both their addresses at the same time. More generally, if a unification adds information to the value of some path π, then that unification simultaneously updates the value of any path that is equivalent to π.

Structure sharing can also be stated using variables such as ?x.

In [51]:
fs1 = nltk.FeatStruct("[ADDRESS1 = [NUMBER =74, STREE = 'RUE PASCAL']]")
fs2 = nltk.FeatStruct("[ADDRESS1 =?x, ADDRESS2 = ?x ]")
print(fs2)

[ ADDRESS1 = ?x ]
[ ADDRESS2 = ?x ]


In [52]:
print(fs1.unify(fs2))

[ ADDRESS1 = (1) [ NUMBER = 74           ] ]
[                [ STREE  = 'RUE PASCAL' ] ]
[                                          ]
[ ADDRESS2 -> (1)                          ]


### Extending a Feature Based Grammar

##### Subcategorization

In 8., we augmented our category labels to represent different kinds of verb, and used the labels IV and TV for intransitive and transitive verbs respectively. This allowed us to write productions like the following:

(27)		
VP -> IV<br>
VP -> TV NP

Although we know that IV and TV are two kinds of V, they are just atomic nonterminal symbols from a CFG, as distinct from each other as any other pair of symbols. This notation doesn't let us say anything about verbs in general, e.g. we cannot say "All lexical items of category V can be marked for tense", since walk, say, is an item of category IV, not V. So, can we replace category labels such as  TV and IV by V along with a feature that tells us whether the verb combines with a following NP object or whether it can occur without any complement?

In [66]:
string = 'play a song on itunes'
tokens= nltk.word_tokenize(string)
tags= nltk.pos_tag(tokens)
print(nltk.ne_chunk(tags).draw())

None
